# TF的MNIST提高

TF是一个非常强大的用来做大规模数值计算的库。其所擅长的任务之一就是实现以及训练深度神经网络。

本文将讲述构建一个TF模型的基本步骤，并通过这些步骤为MNIST构建一个深度卷积神经网络。

若读者还不清楚神经网络和MNIST，请参考[基础篇](./TF的MNIST入门.ipynb)。

## 加载MNIST数据

为方便起见，TF已经准备好脚本来自动下载和倒入MNIST数据集，它会自动创建一个名为MNIST_data的目录存放数据。

In [18]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


这里，mnist是一个轻量级的类，它以NumPy数组的形式存储用于训练、教研、和测试的数据集，同时提供用于在迭代中获取minibatch的函数，后边会用到。

## 运行TF的交互式会话（InteractiveSession）

TF依赖一个高效的C++后端来做计算，与后端的连接叫做Session。一般而言，使用TF的流程先创建一个图，然后在Session中启动它。

这里使用更加方便的InteractiveSession类。通过它，可以更加灵活地构建代码，它能在运行图时插入一些计算图，这些计算图由某些操作构成。这对如IPython等交互式环境来说非常便利。若没有使用InteractiveSession，则需要在启动Session之前构建整个计算图，然后启动之。

In [19]:
import tensorflow as tf

sess = tf.InteractiveSession()

## 计算图

为了在Python中进行高效的数值计算，会用到像NumPy这样的库，将诸如矩阵乘法等耗时操作在Python环境的外部进行，这些计算通常会通过别的语言且更高效的代码实现。

很遗憾的是，每一个操作切换回Python环境仍然开销不小。如果想在GPU或者分布式环境中计算，这一开销更加恐怖，主要原因可能是数据迁移。

TF也是在Python外部完成其主要工作，为了避免上述开销，并没有采用在Python外部独立运行某个耗时操作，而是让设计者事先描述好交互式操作图，然后将其完全运行在Python外部，这与Theano/Torch的做法类似。

因此Python代码的目的是用来构建这个可以在外部运行的计算图、安排计算图的哪一部分应该被运行，参考[学前知识](./TF的套路.ipynb)。

## 构建Softmax回归模型

现在将建立一个拥有一个线性层的Softmax回归模型，后边还会将其扩展为一个拥有多层卷积神经网络的Softmax回归模型。

### 占位符

通过为输入图像和目标输出类别创建节点来开始构建计算图。

In [20]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

这里的x和y并不是特定的值，相反，都只是一个占位符，可以在TF运行某一计算时根据该占位符输入具体的值。

输入图片x是一个2维的浮点数张量，形状是[None, 784]，其中784是一张展开的MNIST图片的维度，None表示大小不定，作为第一个维度，用以指代batch的大小，意为x的数量不定。

输出类别y_也是一个2维张量，其中每一行是一个10维德One-Hot向量，用于代表对应某一MNIST图片的类别。

虽然placeholder的shape的参数是可选的，但又了它，TF能够自动捕捉因数据维度不一致导致的错误。

### 变量

现在为模型定义权重W和偏置b。可以当作额外的输入量，但TF有个更好的处理方式：变量（Variable）。

一个变量代表TF计算图中的一个值，能够在计算过程中使用、甚至修改。在机器学习的应用过程中，模型参数一般用Variable表示。

In [21]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

在调用tf.Variable的时候传入初始值，上例中，把W和b都初始化为全零。

- W是784行10列的矩阵，因为有784个特征和10个输出值
- b是一个10维德向量，因为有10个分类

会话在使用变量之前，变量必须被该会话初始化。这一步真正将指定的初始化值赋给变量，并分配给各个变量，可以一次性初始化全部变量。

In [22]:
sess.run(tf.global_variables_initializer())

### 类别预测和损失函数

现在可以实现Softmax回归函数了，只需要一行！把向量化的图片x和权重矩阵W相乘、加上偏置b，然后计算每个分类的softmax概率。

In [23]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

可以很容易为训练过程制定最小化误差用的损失函数，这里采用目标类别和预测类别之前的交叉熵。

In [24]:
ce = - tf.reduce_sum(y_ * tf.log(y))

注意，tf.reduce_sum把minibatch里的每张图片的交叉熵都加了起来，得到的是整个minibatch的交叉熵。

### 训练模型

已经定义好模型和训练用的损失函数，那么用TF进行训练就很简单了。因为TF知道整个计算图，它通过自动微分法找到对于各个变量的损失梯度值。这里用最速梯度下降法让交叉熵下降，步长是0.01。

In [25]:
trains = tf.train.GradientDescentOptimizer(0.01).minimize(ce)

这一行代码实际上是用来往计算图上添加一个新操作，其中包括计算梯度、计算每个参数的步长变化，并计算出新的参数值。

返回的trains操作对象，在运行时会用梯度下降来更新参数。因此，整个模型的训练可以通过反复地执行trains来完成。

In [26]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    trains.run(feed_dict={x: batch[0], y_: batch[1]})

每一步迭代，都会加载50个训练样本，然后执行一次trains并通过feed_dict将x和y_张量占位符用训练数据替代。

注意：在计算图中，可以用feed_dict来替代任何张量，并不仅限于替换占位符。

### 评估模型

训练出来的模型性能如何呢？

先找出预测正确的标签。tf.argmax是非常有用的函数，它能给出某个张量对象在某一维上的最大数据所在的索引。

由于标签向量由0和1组成，因此最大值1所在的索引位置就是类别标签，比如tf.argmax(y, 1)返回的是模型对于任一输入x预测到的标签值，而tf.argmax(y_, 1)代表正确的标签，通过tf.equal来检测预测和真是标签匹配否（索引一致表示匹配）。

In [27]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

这里返回一个布尔数组，为了计算分类的准确率，将布尔值转换为浮点数代表对、错，然后取平均值。例如：[True, False, True, True]变为[1.0, 0.0, 1.0, 1.0]，计算出平均值为0.75。

In [28]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

最后，计算出在测试数据集上的准确率，大概是0.91。

In [29]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9128


## 构建一个多层卷积神经网络

在MNIST上只有0.91的正确率，实在太糟糕！

下面将用一个稍微复杂的模型——卷积神经网络——来改善效果，正确率会提高到0.992，虽然不是最高，但还是比较令人满意的。

### 权重初始化

为了创建这个模型，需要创建大量的权重和偏置。

这个模型中的权重在初始化时应该加入少量噪声来打破对称性来避免零梯度。

由于使用ReLU神经元，因此比较好的做法是用一个较小的正整数来初始化偏置，以避免神经元出现输出恒为零的问题（dead neurons）。

为了不在建立模型时反复做初始化操作，定义两个函数用于初始化。

In [30]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### 卷积和池化

执行卷积和池化TF的灵活性很高。

怎么处理边界？

步长应设多大？

本例会一直使用vanilla版本：

- 一步长
- 零边距
- 输入输出同尺寸
- 采用简单传统的2*2模板做最大池化

抽象成函数：

In [31]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

#### 第一层卷积

由一个卷积接一个最大池化组成。

卷积在每个5*5的patch中算出32个特征，卷积的权重张量形状是[5, 5, 1, 32]，前两个维度是patch的尺寸，接着是输入的通道数目，最后是输出的通道数据。

而对于每一个输出通道都有一个对应的偏置。

In [34]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

为了用这一层，把x变成一个四维向量，其中第二维、第三维对应图片的宽、高，最后一维代表图片的颜色通道数目（因为是灰度图通道数据为一，若是RGB彩色图，则是三）。

In [35]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

把x_image和权值张量进行卷积，加上偏置，然后应用ReLU激活函数，最后做最大池化。

In [38]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#### 第二层卷积

为了构建更深的网络，会把几个类似的层堆叠起来。这一层中，每个5*5的patch会得到64个特征。

In [39]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#### 密集连接层

现在图片尺寸减小到7*7，假如一个有1024个神经元的全连接层，用于处理整个图片。把池化层输出的张量重塑成一些向量，乘以权重矩阵，加上偏置，然后应用ReLU。

In [41]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#### Dropout

为了减少过拟合，在输出层之前假如dropout。用一个占位符莱代表一个神经元输出在dropout中保持不变的概率，这样可以在训练过程中启用dropout，在测试过程中关闭dropout。TF的tf.nn.dropout操作除了可以屏蔽神经元的输出外，还会自动处理神经元输出的数值范围，所以用dropout的时候可以不考虑数值范围。

In [42]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#### 输出层

最后，添加一个Softmax层，就像之前的单层Softmax回归一样。

In [43]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### 训练和模型评估

为了进行训练和评估，使用与之前简单的单层Softmax神经网络模型几乎一样，只是会用更加复杂的ADAM优化器来做梯度最速下降，在feed_dict中假如额外的参数keep_prob来控制dropout的比例，每迭代100次输出一次日志。

In [45]:
ce = - tf.reduce_sum(y_ * tf.log(y_conv))
trains = tf.train.AdamOptimizer(1e-4).minimize(ce)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step", i, "training accuracy", train_accuracy)
    trains.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print("test accuracy", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0 training accuracy 0.06
step 100 training accuracy 0.84
step 200 training accuracy 0.88
step 300 training accuracy 0.92
step 400 training accuracy 0.92
step 500 training accuracy 0.98
step 600 training accuracy 0.94
step 700 training accuracy 0.94
step 800 training accuracy 0.96
step 900 training accuracy 0.96
step 1000 training accuracy 0.88
step 1100 training accuracy 0.94
step 1200 training accuracy 0.96
step 1300 training accuracy 1.0
step 1400 training accuracy 1.0
step 1500 training accuracy 1.0
step 1600 training accuracy 0.92
step 1700 training accuracy 0.98
step 1800 training accuracy 0.96
step 1900 training accuracy 1.0
step 2000 training accuracy 0.98
step 2100 training accuracy 0.92
step 2200 training accuracy 0.94
step 2300 training accuracy 0.98
step 2400 training accuracy 0.98
step 2500 training accuracy 0.98
step 2600 training accuracy 0.88
step 2700 training accuracy 0.92
step 2800 training accuracy 0.98
step 2900 training accuracy 0.96
step 3000 training accurac

以上代码最终测试集上准确率大概是0.992。

截至目前，已经学会了TF快捷地搭建、训练和评估一个稍微复杂的深度学习模型了吗？